# About this notebook:
- We will be using articles IDs to collect each articles' information, including:
    - Title
    - Abstract
    - Given Mesh Terms
    - Publication Date
- This notebook is part of a series of notebooks under 'Step 2 Articles Collection', where each notebook contain articles published in a particular year.

# Import Libraries and Articles' IDs

In [1]:
# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from Bio import Entrez
import xml.etree.ElementTree as ET

import time
from tqdm import tqdm

# Set the email address associated with your NCBI account
Entrez.email = "geok1723@gmail.com"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df=pd.read_csv("../Step_1_ID_Collection/IDs_2012.csv")

In [2]:
df.head()

,Date_str,ID,Date,Month,Year
0,2012/01/10,22228398,2012-01-10,1,2012
1,2012/01/20,22258900,2012-01-20,1,2012
2,2012/01/12,22233151,2012-01-12,1,2012
3,2012/01/24,22267720,2012-01-24,1,2012
4,2012/01/24,22265392,2012-01-24,1,2012


# Collecting Articles in groups of 100<br>
> The IDs are split into mini-dataframes containing 100 IDs for scraping. <br>
> Output will contain about 360 mini dataframes of articles that are published in this particular year. <br>
> This process was executed using Kaggle, and resulting dataframes are stored in the respective zip-folder. <br>

In [3]:
# determine the group number for each row
group_num = (df.index // 100) + 1

# map the group number to a group label (G1 to G6)
group_label = 'G' + group_num.astype(str)

# add the new column to the DataFrame
df['sub_group'] = group_label

# print(df)
# print(df['sub_group'].nunique())

df['Date']=pd.to_datetime(df['Date'],yearfirst=True)

# group the DataFrame by "Sub_group"
grouped = df.groupby('sub_group')

# create an empty dictionary to store the smaller DataFrames
df_dict = {}

# loop over the groups and create smaller DataFrames
for subgroup, subgroup_df in grouped:
    # construct the name of the smaller DataFrame
    year = subgroup_df['Year'].iloc[0]
    month = subgroup_df['Date'].iloc[0].strftime('%m')
    df_name = f"df_{year}_{month}_{subgroup}"
    # add the smaller DataFrame to the dictionary
    df_dict[df_name] = subgroup_df.copy()

df_ls = list(df_dict.keys())

df_ls.sort()

In [7]:
output_df = {}
for grp in tqdm(df_ls):
    _df = df_dict[grp]
    _s = _df[['ID','Date_str']] 

    print(f"Creating Dataframe of articles for {grp}")
    abstracts = []
    titles = []
    ids = []
    MeSH = []
    Pub_Date = []

    for index,row in tqdm(_s.iterrows()):	

        pub_date = row['Date_str']
        pmid = row['ID']


        try:
            handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="xml")

            xml_data = handle.read()
            handle.close()

            #obtain abstract and title
            root = ET.fromstring(xml_data)
            title = root.find(".//ArticleTitle").text.strip()
            abst = root.findall(".//AbstractText")

            abstract_text = ""
            for abstract in abst:
                abstract_text += abstract.text

            
            #obtain MeSH
            mesh_terms = ['']
            mesh_list = root.findall(".//MeshHeadingList/MeshHeading")
            for mesh in mesh_list:
                descriptor_name = mesh.find(".//DescriptorName").text
                qualifiers = mesh.findall(".//QualifierName")
                qualifier_names = [qualifier.text for qualifier in qualifiers]
                mesh_terms.append((descriptor_name, qualifier_names))
                
            MeSH.append(mesh_terms)            
            ids.append(pmid)
            abstracts.append(abstract_text)
            titles.append(title)
            Pub_Date.append(pub_date)

        except:
            print(f"Error here in ID{pmid}")

    # Store the data in a pandas dataframe
    data = {"ID": ids, "title": titles,"Pub_Date":Pub_Date, "abstract": abstracts, "MeSH_term":MeSH}
    data_df = pd.DataFrame(data)
    df_name = f"ABSTRACT{grp}"
    output_df[df_name] = data_df
    data_df.to_csv(f"{df_name}.csv", index=False)
    print(output_df.keys())

  0%|          | 0/360 [00:00<?, ?it/s]

Creating Dataframe of articles for df_2012_01_G1



0it [00:00, ?it/s]
1it [00:54, 54.04s/it]

Error here in ID22228398



2it [01:25, 40.90s/it]
3it [01:31, 24.88s/it]
4it [01:34, 16.03s/it]
5it [01:35, 10.89s/it]
6it [01:38,  8.06s/it]

...

dict_keys(['ABSTRACTdf_2012_01_G1', 'ABSTRACTdf_2012_01_G10', 'ABSTRACTdf_2012_01_G11', 'ABSTRACTdf_2012_01_G12', 'ABSTRACTdf_2012_01_G13', 'ABSTRACTdf_2012_01_G14', 'ABSTRACTdf_2012_01_G15', 'ABSTRACTdf_2012_01_G16', 'ABSTRACTdf_2012_01_G17', 'ABSTRACTdf_2012_01_G18', 'ABSTRACTdf_2012_01_G19', 'ABSTRACTdf_2012_01_G2', 'ABSTRACTdf_2012_01_G20', 'ABSTRACTdf_2012_01_G21', 'ABSTRACTdf_2012_01_G22', 'ABSTRACTdf_2012_01_G23', 'ABSTRACTdf_2012_01_G24', 'ABSTRACTdf_2012_01_G25', 'ABSTRACTdf_2012_01_G26', 'ABSTRACTdf_2012_01_G27', 'ABSTRACTdf_2012_01_G28', 'ABSTRACTdf_2012_01_G29', 'ABSTRACTdf_2012_01_G3', 'ABSTRACTdf_2012_01_G30', 'ABSTRACTdf_2012_01_G4', 'ABSTRACTdf_2012_01_G5', 'ABSTRACTdf_2012_01_G6', 'ABSTRACTdf_2012_01_G7', 'ABSTRACTdf_2012_01_G8', 'ABSTRACTdf_2012_01_G9', 'ABSTRACTdf_2012_02_G31', 'ABSTRACTdf_2012_02_G32', 'ABSTRACTdf_2012_02_G33', 'ABSTRACTdf_2012_02_G34', 'ABSTRACTdf_2012_02_G35', 'ABSTRACTdf_2012_02_G36', 'ABSTRACTdf_2012_02_G37', 'ABSTRACTdf_2012_02_G38', 'ABSTRACTd


0it [00:00, ?it/s]
1it [00:00,  2.27it/s]
2it [00:00,  2.19it/s]
3it [00:01,  2.22it/s]
4it [00:01,  2.08it/s]
5it [00:02,  2.13it/s]
6it [00:02,  2.09it/s]
7it [00:03,  2.16it/s]
8it [00:03,  2.17it/s]
9it [00:04,  1.95it/s]
10it [00:04,  1.88it/s]
11it [00:05,  1.91it/s]
12it [00:05,  1.99it/s]
13it [00:06,  2.07it/s]
14it [00:06,  2.14it/s]
15it [00:07,  2.02it/s]
16it [00:07,  2.10it/s]
17it [00:08,  2.11it/s]
18it [00:08,  2.07it/s]
19it [00:09,  2.10it/s]
20it [00:09,  2.07it/s]
21it [00:10,  2.02it/s]
22it [00:10,  2.06it/s]
23it [00:11,  2.05it/s]
24it [00:11,  2.13it/s]
25it [00:12,  2.16it/s]
26it [00:12,  1.99it/s]
27it [00:13,  2.02it/s]
28it [00:13,  2.00it/s]
29it [00:14,  2.08it/s]
30it [00:14,  2.01it/s]
31it [00:15,  2.02it/s]
32it [00:15,  2.00it/s]
33it [00:16,  2.10it/s]
34it [00:16,  2.16it/s]
35it [00:17,  1.66it/s]
36it [00:17,  1.76it/s]
37it [00:18,  1.83it/s]
38it [00:18,  1.87it/s]
39it [00:19,  1.90it/s]
40it [00:19,  2.01it/s]
41it [00:20,  2.11it/s]
42it 

dict_keys(['ABSTRACTdf_2012_01_G1', 'ABSTRACTdf_2012_01_G10', 'ABSTRACTdf_2012_01_G11', 'ABSTRACTdf_2012_01_G12', 'ABSTRACTdf_2012_01_G13', 'ABSTRACTdf_2012_01_G14', 'ABSTRACTdf_2012_01_G15', 'ABSTRACTdf_2012_01_G16', 'ABSTRACTdf_2012_01_G17', 'ABSTRACTdf_2012_01_G18', 'ABSTRACTdf_2012_01_G19', 'ABSTRACTdf_2012_01_G2', 'ABSTRACTdf_2012_01_G20', 'ABSTRACTdf_2012_01_G21', 'ABSTRACTdf_2012_01_G22', 'ABSTRACTdf_2012_01_G23', 'ABSTRACTdf_2012_01_G24', 'ABSTRACTdf_2012_01_G25', 'ABSTRACTdf_2012_01_G26', 'ABSTRACTdf_2012_01_G27', 'ABSTRACTdf_2012_01_G28', 'ABSTRACTdf_2012_01_G29', 'ABSTRACTdf_2012_01_G3', 'ABSTRACTdf_2012_01_G30', 'ABSTRACTdf_2012_01_G4', 'ABSTRACTdf_2012_01_G5', 'ABSTRACTdf_2012_01_G6', 'ABSTRACTdf_2012_01_G7', 'ABSTRACTdf_2012_01_G8', 'ABSTRACTdf_2012_01_G9', 'ABSTRACTdf_2012_02_G31', 'ABSTRACTdf_2012_02_G32', 'ABSTRACTdf_2012_02_G33', 'ABSTRACTdf_2012_02_G34', 'ABSTRACTdf_2012_02_G35', 'ABSTRACTdf_2012_02_G36', 'ABSTRACTdf_2012_02_G37', 'ABSTRACTdf_2012_02_G38', 'ABSTRACTd


0it [00:00, ?it/s]
1it [00:00,  2.32it/s]
2it [00:00,  2.30it/s]
3it [00:01,  2.17it/s]
4it [00:01,  2.22it/s]
5it [00:02,  2.23it/s]
6it [00:02,  2.24it/s]
7it [00:03,  2.28it/s]
8it [00:03,  2.30it/s]
9it [00:04,  2.19it/s]
10it [00:04,  2.14it/s]
11it [00:04,  2.20it/s]
12it [00:05,  2.12it/s]
13it [00:05,  2.06it/s]
14it [00:06,  2.04it/s]
15it [00:06,  2.06it/s]
16it [00:07,  2.13it/s]
17it [00:07,  2.18it/s]
18it [00:08,  2.22it/s]
19it [00:08,  2.15it/s]
20it [00:09,  2.20it/s]
21it [00:09,  2.23it/s]
22it [00:10,  2.24it/s]
23it [00:10,  2.26it/s]
24it [00:11,  2.12it/s]
25it [00:11,  2.19it/s]
26it [00:11,  2.23it/s]
27it [00:12,  2.26it/s]
28it [00:12,  2.14it/s]
29it [00:13,  2.01it/s]
30it [00:13,  2.01it/s]
31it [00:14,  2.10it/s]
32it [00:14,  2.08it/s]
33it [00:15,  2.15it/s]
34it [00:15,  2.22it/s]
35it [00:16,  2.26it/s]
36it [00:16,  2.28it/s]
37it [00:16,  2.29it/s]
38it [00:17,  2.28it/s]
39it [00:17,  2.29it/s]
40it [00:18,  2.23it/s]
41it [00:18,  2.27it/s]
42it 

dict_keys(['ABSTRACTdf_2012_01_G1', 'ABSTRACTdf_2012_01_G10', 'ABSTRACTdf_2012_01_G11', 'ABSTRACTdf_2012_01_G12', 'ABSTRACTdf_2012_01_G13', 'ABSTRACTdf_2012_01_G14', 'ABSTRACTdf_2012_01_G15', 'ABSTRACTdf_2012_01_G16', 'ABSTRACTdf_2012_01_G17', 'ABSTRACTdf_2012_01_G18', 'ABSTRACTdf_2012_01_G19', 'ABSTRACTdf_2012_01_G2', 'ABSTRACTdf_2012_01_G20', 'ABSTRACTdf_2012_01_G21', 'ABSTRACTdf_2012_01_G22', 'ABSTRACTdf_2012_01_G23', 'ABSTRACTdf_2012_01_G24', 'ABSTRACTdf_2012_01_G25', 'ABSTRACTdf_2012_01_G26', 'ABSTRACTdf_2012_01_G27', 'ABSTRACTdf_2012_01_G28', 'ABSTRACTdf_2012_01_G29', 'ABSTRACTdf_2012_01_G3', 'ABSTRACTdf_2012_01_G30', 'ABSTRACTdf_2012_01_G4', 'ABSTRACTdf_2012_01_G5', 'ABSTRACTdf_2012_01_G6', 'ABSTRACTdf_2012_01_G7', 'ABSTRACTdf_2012_01_G8', 'ABSTRACTdf_2012_01_G9', 'ABSTRACTdf_2012_02_G31', 'ABSTRACTdf_2012_02_G32', 'ABSTRACTdf_2012_02_G33', 'ABSTRACTdf_2012_02_G34', 'ABSTRACTdf_2012_02_G35', 'ABSTRACTdf_2012_02_G36', 'ABSTRACTdf_2012_02_G37', 'ABSTRACTdf_2012_02_G38', 'ABSTRACTd